<a href="https://colab.research.google.com/github/2IS239-Data-Analytics/Code_along_1/blob/main/Demo%201-%20tva%CC%88tta%20data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Demo - tvätta data med Pandas och Numpy

Klick på Kernel och välj restart and Clear Output om ni vill ta bort all output och testa att köra koden själva. 

Beroende på hur ni väljer att köra Jupyter kan Pandas och Numpy behöva installeras. Om det inte går att importera i nästa steg så genomför följande installation. 

In [ ]:
pip install pandas

In [1]:
#importera pandas och numpy
import pandas as pd
import numpy as np


In [2]:
url = 'copied_raw_GH_link '
df = pd.read_csv(url)

Ladda det dataset som vi kommer att jobba med. Finns i filen BL-Flickr-Images-Book.csv som är en CSV fil som innehåller information om böcker från the British Library. 

In [ ]:
# Undersöka dataset "huvud", dvs. kolumn rubriken och de första raderna. 
df.head()

In [7]:
# visa kolumner i dataset
df.columns

Index(['Identifier', 'Edition Statement', 'Place of Publication',
       'Date of Publication', 'Publisher', 'Title', 'Author', 'Contributors',
       'Corporate Author', 'Corporate Contributors', 'Former owner',
       'Engraver', 'Issuance type', 'Flickr URL', 'Shelfmarks'],
      dtype='object')

Ta bort kolumner som inte behövs i ett dataset.

In [8]:
# Lägg först till kolumnnamn (labels) som ska tas bort i variabeln to_drop.
to_drop = ['Edition Statement',
           'Corporate Author', 
           'Corporate Contributors', 
           'Former owner',
           'Engraver', 
           'Issuance type', 
           'Shelfmarks']

# funktionen drop används sedan för att ta bort de kolumner som finns i strängen to_drop som skapades ovan. 

df.drop(to_drop, inplace=True, axis=1)
df.head()

# to_drop variabeln som innehåller lista med string med de labels som ska tas bort
# inplace = True, dataframe i variabeln df modifieras istället för att retunera en kopia av denna dataframe. 
# axis används för att tala om vad som ska tas bort 
# axis=0 tar bort rader, axis = 1 tar bort kolumner, baserat på de labels som anges.
# flera drop-funktioner: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html

,Identifier,Place of Publication,Date of Publication,Publisher,Title,Author,Contributors,Flickr URL
0,206,London,1879 [1878],S. Tinsley & Co.,Walter Forbes. [A novel.] By A. A,A. A.,"FORBES, Walter.",http://www.flickr.com/photos/britishlibrary/ta...
1,216,London; Virtue & Yorston,1868,Virtue & Co.,All for Greed. [A novel. The dedication signed...,"A., A. A.","BLAZE DE BURY, Marie Pauline Rose - Baroness",http://www.flickr.com/photos/britishlibrary/ta...
2,218,London,1869,"Bradbury, Evans & Co.",Love the Avenger. By the author of “All for Gr...,"A., A. A.","BLAZE DE BURY, Marie Pauline Rose - Baroness",http://www.flickr.com/photos/britishlibrary/ta...
3,472,London,1851,James Darling,"Welsh Sketches, chiefly ecclesiastical, to the...","A., E. S.","Appleyard, Ernest Silvanus.",http://www.flickr.com/photos/britishlibrary/ta...
4,480,London,1857,Wertheim & Macintosh,"[The World in which I live, and my place in it...","A., E. S.","BROOME, John Henry.",http://www.flickr.com/photos/britishlibrary/ta...


Index (första kolumnen, se ovan) genereras automatiskt av Panda när en csv fil laddas, detta för att panda inte vet på föväg vilken kolumn som ska användas som index. Nu vill vi ändra vilken kolumn som ska användas som identifierare till 'Identifier' men först behöver undersöka om raderna i kolumnen innehåller unika värden. 

In [9]:
# funktionen nedan visar "innehållet" i en kolumn.
# de fem första raderna och fem sista visas, även info om hur många rader, 8287 rader finns det i detta dataset. 
df['Identifier']

0           206
1           216
2           218
3           472
4           480
         ...   
8282    4158088
8283    4158128
8284    4159563
8285    4159587
8286    4160339
Name: Identifier, Length: 8287, dtype: int64

In [10]:
# är värderna i kolumnen Identifier unika? dvs. kan vi använda den som identifierare?
df['Identifier'].is_unique

True

Alla värden i kolumnen är unika, så vi bör kunna använda kolumnen som unik identifierare. 

In [11]:
# byta ut det autogenererade index mot Identifier som index. 
# använder set_index funktionen, sätter Identifier till index.
# efter det kan vi använda Identifiervärden som identifierare i vårt dataset
df.set_index('Identifier', inplace=True)
df.head()

,Place of Publication,Date of Publication,Publisher,Title,Author,Contributors,Flickr URL
Identifier,,,,,,,
206,London,1879 [1878],S. Tinsley & Co.,Walter Forbes. [A novel.] By A. A,A. A.,"FORBES, Walter.",http://www.flickr.com/photos/britishlibrary/ta...
216,London; Virtue & Yorston,1868,Virtue & Co.,All for Greed. [A novel. The dedication signed...,"A., A. A.","BLAZE DE BURY, Marie Pauline Rose - Baroness",http://www.flickr.com/photos/britishlibrary/ta...
218,London,1869,"Bradbury, Evans & Co.",Love the Avenger. By the author of “All for Gr...,"A., A. A.","BLAZE DE BURY, Marie Pauline Rose - Baroness",http://www.flickr.com/photos/britishlibrary/ta...
472,London,1851,James Darling,"Welsh Sketches, chiefly ecclesiastical, to the...","A., E. S.","Appleyard, Ernest Silvanus.",http://www.flickr.com/photos/britishlibrary/ta...
480,London,1857,Wertheim & Macintosh,"[The World in which I live, and my place in it...","A., E. S.","BROOME, John Henry.",http://www.flickr.com/photos/britishlibrary/ta...


Visa enskilda objekt (rader). 

In [12]:
# df.loc[206] ger oss objektet 206

df.loc[206]

Place of Publication                                               London
Date of Publication                                           1879 [1878]
Publisher                                                S. Tinsley & Co.
Title                                   Walter Forbes. [A novel.] By A. A
Author                                                              A. A.
Contributors                                              FORBES, Walter.
Flickr URL              http://www.flickr.com/photos/britishlibrary/ta...
Name: 206, dtype: object

In [13]:
# För att visa ett register/objekt/rad baserat på position används iloc funktionen.

df.iloc[0]

Place of Publication                                               London
Date of Publication                                           1879 [1878]
Publisher                                                S. Tinsley & Co.
Title                                   Walter Forbes. [A novel.] By A. A
Author                                                              A. A.
Contributors                                              FORBES, Walter.
Flickr URL              http://www.flickr.com/photos/britishlibrary/ta...
Name: 206, dtype: object

Tvätta specifika kolumner och konvertera dem till ett enhetligt format.

In [14]:
# Visa alla fält från index 1905 i kolumn 'Date of Publication'.

df.loc[1905:, 'Date of Publication']



Identifier
1905              1888
1929       1839, 38-54
2836              1897
2854              1865
2956           1860-63
              ...     
4158088           1838
4158128       1831, 32
4159563      [1806]-22
4159587           1834
4160339        1834-43
Name: Date of Publication, Length: 8275, dtype: object

Här ser vi ett antal olika sätt att skriva datumet för när ett verk publicerades. Några med enbart årtal,
någon med bracket, osv. Här kan vi välja att göra ett antal saker. T.ex. ta bort brackets, osv. Enklast är kanske att välja ut första instansen av fyra påvarandra följande tal och säga att det är publikationsåret. 

För att genomföra det använder vi en s.k. regular expression (r'^(\d{4}).  Uttrycket beskriver ett mönster av fyra påvarandra följande tal. str.extract läser varje cell, extrahera fyra siffror, sätt cellen till det värdet. Här väljer vi att skapa en ny variabel extr och genomföra tvätt av data och spara detta i variabeln för att sedan kunna kontrollera att tvättningen blivit korrekt innan vi implementerar den i vårt dataframe df. 

In [15]:
extr = df['Date of Publication'].str.extract(r'^(\d{4})', expand=False)

# Visa resultatet. 
extr.loc[1905:]

Identifier
1905       1888
1929       1839
2836       1897
2854       1865
2956       1860
           ... 
4158088    1838
4158128    1831
4159563     NaN
4159587    1834
4160339    1834
Name: Date of Publication, Length: 8275, dtype: object

In [16]:
# Obs, variabeln df innehåller fortfarande den orensade varianten.

df['Date of Publication']

Identifier
206        1879 [1878]
216               1868
218               1869
472               1851
480               1857
              ...     
4158088           1838
4158128       1831, 32
4159563      [1806]-22
4159587           1834
4160339        1834-43
Name: Date of Publication, Length: 8287, dtype: object

I och med att resultatet som variablen extr visade är tillfredställande så genomför vi samma tvätt av Date of publication i df. 

In [17]:
df['Date of Publication'] = df['Date of Publication'].str.extract(r'^(\d{4})', expand=False)

In [18]:
# Kontrollera reslutatet
df['Date of Publication']

Identifier
206        1879
216        1868
218        1869
472        1851
480        1857
           ... 
4158088    1838
4158128    1831
4159563     NaN
4159587    1834
4160339    1834
Name: Date of Publication, Length: 8287, dtype: object

Observera att NaN (not a number) finns fortfarande. För att undersöka hur många det finns i vårt data set kan isnull().sum() användas. Det retunerar antalet celler som är NaN (null).
len(df) kan sedan användas för att ta reda på antalet rader i dataframe df.

In [19]:
a = df['Date of Publication'].isnull().sum()
b = len(df)
print(a)
print(b)

971
8287


In [20]:
# Antalet null delat på antalet rader gånger 100 ger hur stor andel av cellerna som är null i procent. 
# Baserat på detta kan vi sedan betämma om vi ska ta bort dessa rader eller låta det vara. 
df['Date of Publication'].isnull().sum()/len(df)*100

11.717147339205985

In [21]:
# visa de 10 första raderna i kolumnen Place of Publication
df['Place of Publication'].head(10)

Identifier
206                                  London
216                London; Virtue & Yorston
218                                  London
472                                  London
480                                  London
481                                  London
519                                  London
667     pp. 40. G. Bryan & Co: Oxford, 1898
874                                 London]
1143                                 London
Name: Place of Publication, dtype: object

Här ser vi att publikationsplats skrivs på olika sätt. T.ex. 'London',  'London; Virtue & Yorston' 

In [22]:
# Visa en specifik rad. 
df.loc[4157862]

Place of Publication                                  Newcastle-upon-Tyne
Date of Publication                                                  1867
Publisher                                                      T. Fordyce
Title                   Local Records; or, Historical Register of rema...
Author                      FORDYCE, T. - Printer, of Newcastle-upon-Tyne
Contributors             SYKES, John - Bookseller, of Newcastle-upon-Tyne
Flickr URL              http://www.flickr.com/photos/britishlibrary/ta...
Name: 4157862, dtype: object

In [23]:
df.loc[4159587]

Place of Publication                                  Newcastle upon Tyne
Date of Publication                                                  1834
Publisher                                                Mackenzie & Dent
Title                   An historical, topographical and descriptive v...
Author                                              Mackenzie, E. (Eneas)
Contributors                                         ROSS, M. - of Durham
Flickr URL              http://www.flickr.com/photos/britishlibrary/ta...
Name: 4159587, dtype: object

Ett annat exempel där Newcastle upon Tyne skrivs på olika sätt. 

Vad ska vi ha för strategi för att tvätta data i kolumnen? 

En strategi är att söka efter där London finns med i texten och ersätta det med enbart strängen London, samma för Oxford. Vi kan också ersätta bindelstreck med vita tecken. Vi kan genomföra detta med hjälp av Numpy genom att skriva följande: 

In [24]:
df['Place of Publication'] = np.where(df['Place of Publication'].str.contains('London'), 'London',
                                np.where(df['Place of Publication'].str.contains('Oxford'), 'Oxford',
                                  df['Place of Publication'].str.replace('-', ' ') ) )
# Söker efter London och ersätter strängen med London. 
# Söker efter Oxford och ersätter strängen med Oxford. 
# Söker efter bindelsträck och ersätter med mellanslag. 

In [25]:
# kontrollera resultatet vi avgränsar till att visa de tio första raderna. 
df.head(10)

,Place of Publication,Date of Publication,Publisher,Title,Author,Contributors,Flickr URL
Identifier,,,,,,,
206,London,1879,S. Tinsley & Co.,Walter Forbes. [A novel.] By A. A,A. A.,"FORBES, Walter.",http://www.flickr.com/photos/britishlibrary/ta...
216,London,1868,Virtue & Co.,All for Greed. [A novel. The dedication signed...,"A., A. A.","BLAZE DE BURY, Marie Pauline Rose - Baroness",http://www.flickr.com/photos/britishlibrary/ta...
218,London,1869,"Bradbury, Evans & Co.",Love the Avenger. By the author of “All for Gr...,"A., A. A.","BLAZE DE BURY, Marie Pauline Rose - Baroness",http://www.flickr.com/photos/britishlibrary/ta...
472,London,1851,James Darling,"Welsh Sketches, chiefly ecclesiastical, to the...","A., E. S.","Appleyard, Ernest Silvanus.",http://www.flickr.com/photos/britishlibrary/ta...
480,London,1857,Wertheim & Macintosh,"[The World in which I live, and my place in it...","A., E. S.","BROOME, John Henry.",http://www.flickr.com/photos/britishlibrary/ta...
481,London,1875,William Macintosh,"[The World in which I live, and my place in it...","A., E. S.","BROOME, John Henry.",http://www.flickr.com/photos/britishlibrary/ta...
519,London,1872,The Author,Lagonells. By the author of Darmayne (F. E. A....,"A., F. E.","ASHLEY, Florence Emily.",http://www.flickr.com/photos/britishlibrary/ta...
667,Oxford,NaN,NaN,"The Coming of Spring, and other poems. By J. A...","A., J.|A., J.","ANDREWS, J. - Writer of Verse",http://www.flickr.com/photos/britishlibrary/ta...
874,London,1676,NaN,"A Warning to the inhabitants of England, and L...",Remaʿ.,"ADAMS, Mary.",http://www.flickr.com/photos/britishlibrary/ta...


In [26]:
# kontollera resulatet för objekt 4157862 som innehöll - i publikationsplats. 
df.loc[4157862]

Place of Publication                                  Newcastle upon Tyne
Date of Publication                                                  1867
Publisher                                                      T. Fordyce
Title                   Local Records; or, Historical Register of rema...
Author                      FORDYCE, T. - Printer, of Newcastle-upon-Tyne
Contributors             SYKES, John - Bookseller, of Newcastle-upon-Tyne
Flickr URL              http://www.flickr.com/photos/britishlibrary/ta...
Name: 4157862, dtype: object

Om vi nu kontrollerar alla unika värden i kolumnen Place of Publication så ser vi att det finns mer som behöver tvättas. 

In [27]:
# Visa de 40 första unika värdena i kolumnen 'Place of Publication'. Vi ser att det finns mer som behöver tvättas. 
pd.Series(df['Place of Publication'].unique())[:40]

0                                 London
1                                 Oxford
2                               Coventry
3                            Christiania
4                                Firenze
5                              Amsterdam
6                                 Savona
7                                  Paris
8                            Puerto Rico
9                               New York
10                                  Hull
11                                Oxonii
12                                Milano
13                              Aberdeen
14                                  Wien
15                              Abingdon
16                                Quebec
17                               Leipzig
18    pp. 40. W. Cann: Plymouth, [1876?]
19                                  1845
20                                   enk
21                                Boston
22                               Upsaliæ
23                             Edinburgh
24              

## Nytt dataset
Textfil med universitetsstäder. 


In [28]:
uni_towns = open(path +'/university_towns.txt')
uni_towns.readlines()[:20]

['Alabama[edit]\n',
 'Auburn (Auburn University)[1]\n',
 'Florence (University of North Alabama)\n',
 'Jacksonville (Jacksonville State University)[2]\n',
 'Livingston (University of West Alabama)[2]\n',
 'Montevallo (University of Montevallo)[2]\n',
 'Troy (Troy University)[2]\n',
 'Tuscaloosa (University of Alabama, Stillman College, Shelton State)[3][4]\n',
 'Tuskegee (Tuskegee University)[5]\n',
 'Alaska[edit]\n',
 'Fairbanks (University of Alaska Fairbanks)[2]\n',
 'Arizona[edit]\n',
 'Flagstaff (Northern Arizona University)[6]\n',
 'Tempe (Arizona State University)\n',
 'Tucson (University of Arizona)\n',
 'Arkansas[edit]\n',
 'Arkadelphia (Henderson State University, Ouachita Baptist University)[2]\n',
 'Conway (Central Baptist College, Hendrix College, University of Central Arkansas)[2]\n',
 'Fayetteville (University of Arkansas)[7]\n',
 'Jonesboro (Arkansas State University)[8]\n']

För att kunna arbeta med ett dataframe och använda Pandas behöver dock textfilens innehåll läsas över (formateras) till en tabell. Hur kan vi göra det? 

Först ser vi delstaten följt av [edit]. Efter det kommer städer följt av namnet på universitetet, dvs. stad och universitet i den aktuella delstaten. För att konvertera till csvformat kan vi använda denna struktur, loopa igenom textfilen och plocka ut den del där Edit finns och använda det som avgränsning. Och sedan skapa en lista med delstat, stad och universitet. 


In [ ]:
uni_towns = []
with open(path +'/university_towns.txt') as file:
    for line in file:
        if'[edit]' in line:
            state = line
        else:
            uni_towns.append((state, line))


# for-loopen går igenom alla rader i filen. Om en rad (line) innehåller [edit] får variabeln state värdet i raden. 
# dvs. om delstat hittas sätts state till den aktuella delstaten. 
# Om en rad inte innehåller [edit] motsvarar line den aktuella raden.
# dvs. line sätts till stad och universitet
# state och line paras ihop och läggs till i uni_towns.
# hela filen loopas igenom på detta sätt

# visa 20 första raderna i uni_towns efter att for-loopen körts. 
uni_towns[:20]

In [30]:
# Lägga till i ett panda dataframe
# Läggs i korrekt kolumn i och med att state och line är ihopparade. I funktionen namnges även kolumnrubrikerna. 
uni_towns_df = pd.DataFrame(uni_towns, columns=['State', 'Region Name'])
uni_towns_df[:20]

,State,Region Name
0,Alabama[edit]\n,Auburn (Auburn University)[1]\n
1,Alabama[edit]\n,Florence (University of North Alabama)\n
2,Alabama[edit]\n,Jacksonville (Jacksonville State University)[2]\n
3,Alabama[edit]\n,Livingston (University of West Alabama)[2]\n
4,Alabama[edit]\n,Montevallo (University of Montevallo)[2]\n
5,Alabama[edit]\n,Troy (Troy University)[2]\n
6,Alabama[edit]\n,"Tuscaloosa (University of Alabama, Stillman Co..."
7,Alabama[edit]\n,Tuskegee (Tuskegee University)[5]\n
8,Alaska[edit]\n,Fairbanks (University of Alaska Fairbanks)[2]\n
9,Arizona[edit]\n,Flagstaff (Northern Arizona University)[6]\n


 Vi behöver tvätta data, vi hade kunnat göra det direkt i for-loopen innan. Men vi kan även göra det genom att först definiera en funktion som undersöker om det finns ett mellanslag och en ( och om det finns bracket


In [31]:

def get_citystate (item):
    if ' (' in item:
        return item[:item.find(' (')]
    elif '[' in item:
        return item[:item.find('[')]
    else:
        return item
    


In [32]:
# Inget har förändrats i vårt dataset, metoden har enbart definierats. 
uni_towns_df.loc[2]

State                                            Alabama[edit]\n
Region Name    Jacksonville (Jacksonville State University)[2]\n
Name: 2, dtype: object

In [33]:
# Metoden retunerar tvättad sträng, state
get_citystate('Wyoming[edit]\n')

'Wyoming'

In [34]:
# metoden retunerar tvättad sträng, region name
get_citystate('Eau Claire (University of Winsconsin-Eau Claire)\n')

'Eau Claire'

In [35]:
# applymap kan sedan användas för att tvätta hela uni_towns_df.
# dvs. funktionen applymap kör funktionen get_citystate för alla celler i df. 

uni_towns_df = uni_towns_df.applymap(get_citystate)
uni_towns_df[:20]

,State,Region Name
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


Nu har vi tvättad data i hela vårt dataset.